# Heston Model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import heston as hes

In [ ]:
from financepy.finutils import *
from financepy.market.curves import *
from financepy.products.equity import *

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interact_manual

In [ ]:
def implied_vol(valueDate, expiryDate, stockPrice, strike, interestRate, dividendYield, call_price):
    discountCurve = FinDiscountCurveFlat(valueDate, interestRate, FinFrequencyTypes.CONTINUOUS)
    callOption = FinEquityVanillaOption(expiryDate, strike, FinOptionTypes.EUROPEAN_CALL)
    return callOption.impliedVolatility(valueDate, stockPrice, discountCurve, dividendYield, call_price)

### Market Data

In [ ]:
stockPrice = 100
t = 1.0
valueDate = FinDate(1, 9, 2020)
expiryDate = valueDate.addYears(t)
strikes = np.linspace(80, 120, 11)
r, d = 0, 0
alpha = 0.75  # carr-madan damping factor

### Call option with Black model using Carr Madan Fourier method

In [ ]:
@interact
def plot_Black_using_Fourier(vol = (0.02, 0.5, 0.02)):     # the volatility at time 0, NOT variance                             
    prices = [hes.carr_madan_black_call_option(stockPrice, vol, t, k, alpha) for k in strikes]      
    vols = [implied_vol(valueDate, expiryDate, stockPrice, k, r, d, p) for (k, p) in zip(strikes, prices)]       
    plt.plot(strikes, vols); 
    plt.ylim((min(vols)*0.9, max(vols)*1.1))
    plt.title("Black"); plt.xlabel("Strike"); plt.ylabel("Black Volatility"); plt.grid()

### Call option with Heston model using Carr Madan Fourier method

In [ ]:
@interact
def plot_Heston(vol_0 = (0.1, 0.9, 0.1),      # the volatility at time 0, NOT variance
                kappa = (0.01, 0.05, 0.01),   # mean reversion rate
                nu = (0.1, 1.0, 0.1),         # mean reversion level
                lam = (0.1, 2, 0.1),        # vol of variance
                rho = (-1, 0.5, 0.1)):          # correlation between the variance and the underlying               
    
    prices = [hes.carr_madan_heston_call_option(stockPrice, vol_0, kappa, nu, lam, rho, t, k, alpha) for k in strikes]  
    vols = [implied_vol(valueDate, expiryDate, stockPrice, k, r, d, p) for (k, p) in zip(strikes, prices)]       
    plt.plot(strikes, vols); 
    plt.ylim((min(vols)*0.9, max(vols)*1.1))
    plt.title("Heston"); plt.xlabel("Strike"); plt.ylabel("Black Volatility"); plt.grid()